In [1]:
%env SUPERSTAQ_API_KEY=fa515f2fff2dad66cc1c0797ee98c66fa275b13ecc86e3ff81e2ce88f280a739

env: SUPERSTAQ_API_KEY=fa515f2fff2dad66cc1c0797ee98c66fa275b13ecc86e3ff81e2ce88f280a739


In [14]:
%env SUPERSTAQ_REMOTE_HOST=https://127.0.0.1:5000

env: SUPERSTAQ_REMOTE_HOST=https://127.0.0.1:5000


In [9]:
import stimcirq
from qldpc import circuits, codes
import cirq_superstaq as css
code = codes.SteaneCode()
circuit = circuits.get_memory_experiment(code, num_rounds=1)
circuit= stimcirq.stim_circuit_to_cirq_circuit(circuit)

In [10]:
circuit

┌───┐   ┌───┐   ┌───┐   ┌──────┐   ┌───┐   ┌───┐   ┌──────────┐
(0, 0): ───RX──────────────────────────────X──────────────────────────@───────────MX('6')───────────────────────
                                           │                          │
(0, 1): ───RX─────────────────────────────X┼─────────────────────────@┼───────────MX('7')───────────────────────
                                          ││                         ││
(0, 2): ───RX──────X──────────────X───────┼┼─@────────@──────────────┼┼───────────MX('8')───────────────────────
                   │              │       ││ │        │              ││
(0, 3): ───RX──────┼──────────────┼──────X┼┼─┼────────┼─────────────@┼┼───────────MX('9')───────────────────────
                   │              │      │││ │        │             │││
(0, 4): ───RX──────┼─────X────────┼X─────┼┼┼@┼────────┼───────@─────┼┼┼───────────MX('10')──────────────────────
                   │     │        ││     │││││        │       │     │││
(0, 5): ───RX────X─┼─────┼X───────┼┼─────┼┼┼┼┼──────@─┼──────@┼─────┼┼┼───────────MX('11')──────────────────────
                 │ │     ││       ││     │││││      │ │      ││     │││
(0, 6): ───RX────┼X┼─────┼┼X─────X┼┼─────┼┼┼┼┼@─────┼@┼─────@┼┼─────┼┼┼───────────MX('12')──────────────────────
                 │││     │││     │││     ││││││     │││     │││     │││
(1, 0): ───RX────@┼┼─────@┼┼─────@┼┼─────@┼┼┼┼┼─────┼┼┼─────┼┼┼─────┼┼┼MX('0')──────────────────────────────────
                  ││      ││      ││      │││││     │││     │││     │││
(1, 1): ───RX─────@┼──────@┼──────@┼──────@┼┼┼┼─────┼┼┼─────┼┼┼─────┼┼┼MX('1')──────────────────────────────────
                   │       │       │       ││││     │││     │││     │││
(1, 2): ───RX──────@───────@───────@───────@┼┼┼─────┼┼┼─────┼┼┼─────┼┼┼MX('2')──────────────────────────────────
                                            │││     │││     │││     │││
(1, 3): ───RX───────────────────────────────@┼┼─────@┼┼─────@┼┼─────@┼┼───────────MX('3')───────────────────────
                                             ││      ││      ││      ││
(1, 4): ───RX────────────────────────────────@┼──────@┼──────@┼──────@┼───────────MX('4')───────────────────────
                                              │       │       │       │
(1, 5): ───RX─────────────────────────────────@───────@───────@───────@───────────MX('5')───────────────────────
                                                                                  Det('0')
                                                                                  Det('1')
                                                                                  Det('2')
                                                                                  Det('0','10','11','12','9')
                                                                                  Det('1','11','12','7','8')
                                                                                  Det('10','12','2','6','8')
                                                                                  Obs0('10','11','8')
                └───┘   └───┘   └───┘   └──────┘   └───┘   └───┘   └──────────┘

In [17]:
service = css.Service()
compiler_output = service.cq_compile(circuit)
print(compiler_output.circuit[0:10])

/home/victory/pyLIQTR/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


0: ──────────────RGate(0.5π, 0)──────────────RGate(0.5π, -0.5π)───────────────RGate(-0.5π, -0.5π)───────RGate(0.5π, -0.5π)───
                 │                           │                                │                         │
1: ──────────────#2──────────────────────────#2───────────────────────────────#2────────────────────────#2───────────────────
                 │                           │                                │                         │
2: ──────────────#3──────────────────────────#3───────────────────────────────#3────────────────────────#3───────────────────
                 │                           │                                │                         │
3: ──────────────#4──────────────────────────#4───────────────────────────────#4────────────────────────#4───────────────────
                 │                           │                                │                         │
4: ──────────────#5──────────────────────────#5─────────────────────────

In [35]:
def custom_resolver(cirq_type: str) -> type[cirq.Gate] | None:
    """Tells `cirq.read_json` how to deserialize `cirq-superstaq`'s custom gates.

    Changes to gate names in this file should be reflected in this resolver.
    See quantumai.google/cirq/dev/serialization for more information                    about (de)serialization.

    Args:
        cirq_type: The string of the gate type for the serializer to resolve.

    Returns:
        The resolved `cirq.Gate` matching the input, or None if no match.
    """
    type_to_gate_map: dict[str, type[cirq.Gate]] = {
        "MeasureAndOrResetGate": stimcirq.MeasureAndOrResetGate,
        "DetAnnotation": stimcirq.DetAnnotation,
        "CumulativeObservableAnnotation": stimcirq.CumulativeObservableAnnotation,
    }
    return type_to_gate_map.get(cirq_type)

NameError: name 'cirq' is not defined

In [22]:
# print(repr(circuit))
import cirq

In [23]:
serialized_circuit = cirq.to_json(circuit)

In [24]:
SUPERSTAQ_RESOLVERS = [
    css.ops.qudit_gates.custom_resolver,
    css.ops.qubit_gates.custom_resolver,
    custom_resolver,
]
resolvers = [*SUPERSTAQ_RESOLVERS, *cirq.DEFAULT_RESOLVERS]
deserialized_obj = cirq.read_json(json_text=serialized_circuit, resolvers=resolvers)

In [ ]:
compiler_output = service.compile(circuit, target="eeroq_wonderlake_qpu")

# Call `.circuit` on the compiler output to get the corresponding output circuit
compiled_circuit = compiler_output.circuit

# Visualize the compiled circuit
print(compiled_circuit)

In [25]:
compiler_output = service.cq_compile(circuit)

SuperstaqServerException: Internal Server Error (Status code: 500)

If you would like to contact a member of our team, email us at superstaq@infleqtion.com or join our Slack workspace: https://join.slack.com/t/superstaq/shared_invite/zt-1wr6eok5j-fMwB7dPEWGG~5S474xGhxw.